In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive

import os

PROJECT_DIR = '/content/drive/MyDrive/afrisent-semeval-2023'

/content/drive/MyDrive


In [3]:
!pip install wget
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=3c88737ab6cf016d7da5e5df237625955d53195c6231337fab890c9276536158
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 32.8 MB/s 
     |████████████████████████████████| 1.2 MB 59.8 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 1.6 MB 57.1 MB/s 
     |████████████████████████████████| 788 kB 58.6 MB/s 
     |████████████████████████████████| 182 kB 71.8 MB/s 
     |████████████████████████████████| 19.7 MB 377 kB/s 
     |████████████████████████████████| 981 kB 68.5 MB/s 
     |████████████████████████████████| 46 kB 4.9 MB/s 
     |████

In [8]:
import wget
import flair
from flair.data import Sentence
from flair.models import TextClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score

Ensure below path is present and download the pre-trained model to this location. Commented it out after the use.

In [ ]:
# wget.download("http://ltdata1.informatik.uni-hamburg.de/amharic/taskmodels/sent/final-model.pt",
#                               out = '/content/drive/MyDrive/afrisent-semeval-2023/pretrained_models/final-model.pt')

'/content/drive/MyDrive/afrisent-semeval-2023/pretrained_models/final-model.pt'

In [5]:
## Load the amharic sentiment pre-trained model
classifier = TextClassifier.load('/content/drive/MyDrive/afrisent-semeval-2023/pretrained_models/final-model.pt')


2022-11-27 20:40:24,879 loading file /content/drive/MyDrive/afrisent-semeval-2023/pretrained_models/final-model.pt


In [7]:
def get_label_flair(text, classifier = classifier):
    """Get text/sentence from the dataframe and use the flair model to predict the label"""
    sentence = Sentence(text)
    classifier.predict(sentence)  ## 'all labels' -- can be []
    all_labels_list = sentence.to_dict()['all labels']
    if len(all_labels_list) == 0 :
        ## can choose one of ['neutral', 'negative', 'positive'] at random
        ## or choose 'neutral' because it was the most common thing, if we aren't sure of sentiment
        label_pred = 'neutral'  ## np.random.choice(['neutral', 'negative', 'positive'], 1)[0]
        return label_pred
    else:
        label_pred = all_labels_list[0]['value'].lower()
        return label_pred

In [9]:
def get_labels(file_path):
    """Get the .tsv file with text and label and give the true and the predicted labels."""
    df = pd.read_csv(file_path, sep='\t')
    y_true = df.label.to_list() ## comment this cell, if the file has only text column
    y_pred = df.iloc[:]['text'].apply(lambda x : get_label_flair(x)).to_list()
    return y_true, y_pred ## remove the y_true variable if the file has only text column

In [10]:
def get_scores(y_true, y_pred):

  """Get true values and predicted values and give the predicted scores"""

  f1_score_macro = f1_score(y_true, y_pred, average='macro')   
  accuracy = accuracy_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred, average=None)
  precision = precision_score(y_true, y_pred, average=None)

  results = {'f1_score_macro' : f1_score_macro,
              'accuracy' : accuracy,
              'recall_score' : recall,
              'precision_score' : precision}
  return results

In [11]:
%cd /content/drive/MyDrive


PROJECT_DIR = '/content/drive/MyDrive/afrisent-semeval-2023'
TASK = 'SubtaskA'
LANGUAGE_CODE = 'am'

/content/drive/MyDrive


### AM test set results

In [ ]:
file_path = os.path.join(PROJECT_DIR, TASK, 'train', 'fil-splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
print('fil-splitted-train-dev-test/am/test.tsv')

/content/drive/MyDrive
fil-splitted-train-dev-test/am/test.tsv


In [ ]:
true_vals, pred_vals = get_labels(file_path = file_path)

In [ ]:
model_results = get_scores(true_vals, pred_vals)
print(model_results)

{'f1_score_macro': 0.6469691387059684, 'accuracy': 0.6862302483069977, 'recall_score': array([0.44351464, 0.86899563, 0.55026455]), 'precision_score': array([0.73103448, 0.65785124, 0.76470588])}


In [ ]:
print(classification_report(true_vals, pred_vals))

              precision    recall  f1-score   support

    negative       0.73      0.44      0.55       239
     neutral       0.66      0.87      0.75       458
    positive       0.76      0.55      0.64       189

    accuracy                           0.69       886
   macro avg       0.72      0.62      0.65       886
weighted avg       0.70      0.69      0.67       886



### AM train set results

In [ ]:
### Train set ###
file_path = os.path.join(PROJECT_DIR, TASK, 'train', 'fil-splitted-train-dev-test', LANGUAGE_CODE, 'train.tsv')

In [ ]:
true_vals, pred_vals = get_labels(file_path = file_path)

In [ ]:
model_results = get_scores(true_vals, pred_vals)
print(model_results)

{'f1_score_macro': 0.6442773553439872, 'accuracy': 0.6848220769789397, 'recall_score': array([0.4376784 , 0.85946197, 0.55844156]), 'precision_score': array([0.65063649, 0.66606758, 0.80373832])}


In [ ]:
print(classification_report(true_vals, pred_vals))

              precision    recall  f1-score   support

    negative       0.65      0.44      0.52      1051
     neutral       0.67      0.86      0.75      2156
    positive       0.80      0.56      0.66       924

    accuracy                           0.68      4131
   macro avg       0.71      0.62      0.64      4131
weighted avg       0.69      0.68      0.67      4131



### AM Dev set results

In [ ]:
### Dev set ###
file_path = os.path.join(PROJECT_DIR, TASK, 'train', 'fil-splitted-train-dev-test', LANGUAGE_CODE, 'dev.tsv')

In [ ]:
true_vals, pred_vals = get_labels(file_path = file_path)

In [ ]:
model_results = get_scores(true_vals, pred_vals)
print(model_results)

{'f1_score_macro': 0.6551927818599541, 'accuracy': 0.6949152542372882, 'recall_score': array([0.43555556, 0.85957447, 0.59473684]), 'precision_score': array([0.66666667, 0.67785235, 0.79577465])}


In [ ]:
print(classification_report(true_vals, pred_vals))

              precision    recall  f1-score   support

    negative       0.67      0.44      0.53       225
     neutral       0.68      0.86      0.76       470
    positive       0.80      0.59      0.68       190

    accuracy                           0.69       885
   macro avg       0.71      0.63      0.66       885
weighted avg       0.70      0.69      0.68       885



### AM pre-processed/filtered dataset

In [ ]:
file_path = os.path.join(PROJECT_DIR, TASK, 'train', 'filtered-train-data', LANGUAGE_CODE, 'am_train_filtered.tsv')

In [ ]:
true_vals, pred_vals = get_labels(file_path = file_path)

In [ ]:
model_results = get_scores(true_vals, pred_vals)
print(model_results)

{'f1_score_macro': 0.6463635611174893, 'accuracy': 0.6865469332429684, 'recall_score': array([0.43828383, 0.86089494, 0.56254797]), 'precision_score': array([0.66466466, 0.66658298, 0.79673913])}


In [ ]:
print(classification_report(true_vals, pred_vals))

              precision    recall  f1-score   support

    negative       0.66      0.44      0.53      1515
     neutral       0.67      0.86      0.75      3084
    positive       0.80      0.56      0.66      1303

    accuracy                           0.69      5902
   macro avg       0.71      0.62      0.65      5902
weighted avg       0.69      0.69      0.67      5902



### AM pre-processed/filtered and balanced dataset

In [12]:
file_path = os.path.join(PROJECT_DIR, TASK, 'train', 'filtered-balanced-train-data', LANGUAGE_CODE, 'am_train_filtered_balanced.tsv')

In [13]:
true_vals, pred_vals = get_labels(file_path = file_path)

In [14]:
model_results = get_scores(true_vals, pred_vals)
print(model_results)

{'f1_score_macro': 0.6181433501246056, 'accuracy': 0.6175492453312867, 'recall_score': array([0.43821949, 0.85188028, 0.56254797]), 'precision_score': array([0.7623498 , 0.47700902, 0.87995198])}


In [15]:
print(classification_report(true_vals, pred_vals))

              precision    recall  f1-score   support

    negative       0.76      0.44      0.56      1303
     neutral       0.48      0.85      0.61      1303
    positive       0.88      0.56      0.69      1303

    accuracy                           0.62      3909
   macro avg       0.71      0.62      0.62      3909
weighted avg       0.71      0.62      0.62      3909

